<a href="https://colab.research.google.com/github/47thtechcorner/RayCodes_AIStudyBuddy/blob/main/RayCodes_AIStudyBuddy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# @title Install Dependencies
# @markdown **Step 1 : Installing and Importing all required libraries**

!pip install -q --upgrade langchain-google-genai langchain-community faiss-cpu pypdf reportlab

from IPython.display import clear_output
import os
import sys
from google.colab import drive
from google.colab import userdata
import getpass
import re
import ast

from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS

from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings

from reportlab.lib.pagesizes import LETTER
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, PageBreak
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib import colors
clear_output()


In [2]:
# @title Drive and Gemini Setup
# @markdown **Step 2 : Connecting to Google Drive and Setting up Gemini Model**

if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')

try:
    GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')
except:
    print("❌ ERROR: Did not find 'GEMINI_API_KEY' in Secrets (🔑).")
    GEMINI_API_KEY = getpass.getpass("Enter your Google Gemini API key: ")

os.environ["GEMINI_API_KEY"] = GEMINI_API_KEY
os.environ["GEMINI_MODEL"] = "gemini-3-flash-preview"
# os.environ["GEMINI_MODEL"] = "gemini-2.5-flash"

Mounted at /content/drive


In [8]:
# @title Preprocess Data
# @markdown **Step 3 : Preprocessing your PDF into a vector store for Gemini (one time run)**

def process_and_store_pdf(file_path):
    """
    Reads a PDF, chunks it, creates embeddings, and saves it
    in a structured format for downstream AI tasks.
    """

    # 1. Setup Paths
    if not os.path.exists(file_path):
        return f"Error: File not found at {file_path}"

    base_name = os.path.splitext(os.path.basename(file_path))[0]
    directory = os.path.dirname(file_path)
    output_folder = os.path.join(directory, f"{base_name}_data")

    # Create the output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    print(f"Processing '{base_name}'...")

    # 2. Read the File (PyPDFLoader handles page numbers well)
    loader = PyPDFLoader(file_path)
    pages = loader.load()
    print(f"Loaded {len(pages)} pages.")

    # 3. Chunk the Text (Critical for RAG/Chatbots)
    # We use a large chunk size with overlap to keep context for math/equations
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200,
        separators=["\n\n", "\n", " ", ""]
    )
    docs = text_splitter.split_documents(pages)
    print(f"Split into {len(docs)} text chunks.")

    # 4. Create Embeddings & Vector Store
    # This converts text to vectors using Gemini's embedding model
    embeddings = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")
    vector_store = FAISS.from_documents(docs, embeddings)

    # 5. Save the Index locally
    # This allows us to reload it later without reprocessing
    index_path = os.path.join(output_folder, "faiss_index")
    vector_store.save_local(index_path)

    print(f"✅ Success! Optimized data stored in: {output_folder}")
    return output_folder

file_path = input("Enter your PDF file location:").strip()
file_path = f"/content/drive/MyDrive/{file_path}.pdf"

base_name = os.path.splitext(os.path.basename(file_path))[0]
directory = os.path.dirname(file_path)
output_folder = os.path.join(directory, f"{base_name}_data")
if os.path.exists(output_folder):
  print("Preprocessed folder already exists! Skipping this step..")
  output_dir = output_folder
else:
  output_dir = process_and_store_pdf(file_path)

Enter your PDF file location:StudyBuddy/physics_chapter
Preprocessed folder already exists! Skipping this step..


In [ ]:
# @title Building Paper
# @markdown **Step 4 : Building the MCQ paper basis your topic**

import os
import re
import ast
from datetime import datetime
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI
from langchain_community.vectorstores import FAISS

# ReportLab Imports for "LaTeX-Look"
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, PageBreak, Frame, PageTemplate
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.pagesizes import A4
from reportlab.lib.units import inch, mm
from reportlab.lib.enums import TA_CENTER, TA_JUSTIFY, TA_LEFT
from reportlab.pdfgen import canvas
from reportlab.graphics.shapes import Drawing, Line

# --- NEW: Helper to convert LaTeX to PDF-friendly text (KEPT AS IS) ---
def clean_latex_to_reportlab(text):
    """
    Converts basic LaTeX math into ReportLab-friendly HTML/Unicode.
    """
    if not text: return ""

    # 1. Common Greek & Math Symbols (Unicode)
    replacements = {
        r'\\omega': 'ω', r'\\alpha': 'α', r'\\beta': 'β', r'\\gamma': 'γ',
        r'\\theta': 'θ', r'\\pi': 'π', r'\\Delta': 'Δ', r'\\sigma': 'σ',
        r'\\tau': 'τ', r'\\lambda': 'λ', r'\\mu': 'μ', r'\\rho': 'ρ',
        r'\\approx': '≈', r'\\times': '×', r'\\cdot': '·', r'\\leq': '≤', r'\\geq': '≥',
        r'\\rightarrow': '→', r'\\infty': '∞'
    }

    for latex, char in replacements.items():
        text = re.sub(latex, char, text)

    # 2. Superscripts: x^2 or x^{2} -> x<sup>2</sup>
    text = re.sub(r'\^\{([^\}]+)\}', r'<sup>\1</sup>', text) # {brackets}
    text = re.sub(r'\^(\d+)', r'<sup>\1</sup>', text)       # single digit

    # 3. Subscripts: x_2 or x_{cm} -> x<sub>cm</sub>
    text = re.sub(r'_\{([^\}]+)\}', r'<sub>\1</sub>', text) # {brackets}
    text = re.sub(r'_([a-zA-Z0-9]+)', r'<sub>\1</sub>', text) # simple chars

    # 4. Remove the $ signs (Italicize the content inside instead)
    def italicize_math(match):
        content = match.group(1)
        return f"<i>{content}</i>"

    text = re.sub(r'\$([^\$]+)\$', italicize_math, text)

    return text

# --- NEW: Formatting Functions for Headers/Footers ---
def header_footer(canvas, doc):
    """
    Draws the Header and Footer on every page to mimic fancyhdr.
    """
    canvas.saveState()

    # --- Header ---
    canvas.setFont('Times-Roman', 10)
    # Left Header
    canvas.drawString(1 * inch, 11.2 * inch, "Test Paper")
    # Right Header
    canvas.drawRightString(7.27 * inch, 11.2 * inch, "AI Generated Exam")
    # Header Line
    canvas.setLineWidth(0.5)
    canvas.line(1 * inch, 11.1 * inch, 7.27 * inch, 11.1 * inch)

    # --- Footer ---
    # Right Footer (Page Number)
    page_num = canvas.getPageNumber()
    canvas.drawRightString(7.27 * inch, 0.75 * inch, f"Page {page_num}")

    canvas.restoreState()

# ---------------------------------------------------------

def generate_professional_mcq(output_dir, topic=None, num_questions=5):
    """
    Generates a professionally formatted MCQ PDF with LaTeX-like styling.
    """

    # 1. Load Data
    if not os.path.exists(output_dir):
        return "Error: output_dir does not exist."

    embeddings = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")
    try:
        vector_store = FAISS.load_local(
            os.path.join(output_dir, "faiss_index"),
            embeddings,
            allow_dangerous_deserialization=True
        )
    except Exception as e:
        return f"Error loading index: {str(e)}"

    # 2. Retrieve Context
    query = topic if topic else "Core concepts, definitions, and equations"
    docs = vector_store.similarity_search(query, k=10)
    context_text = "\n\n".join([d.page_content for d in docs])

    # 3. Generate Content
    model_name = os.environ.get("GEMINI_MODEL")
    if not model_name:
        return "Error: GEMINI_MODEL env var not set."

    llm = ChatGoogleGenerativeAI(model=model_name, temperature=0.3)

    prompt = f"""
    You are a university professor writing a formal exam paper.
    Based on the following syllabus (CONTEXT), write {num_questions} multiple-choice questions.

    CONTEXT:
    {context_text}

    RULES:
    1. Questions must be independent.
    2. Format EXACTLY like this:

       Q#. [Question Text]
       A) [Option A]
       B) [Option B]
       C) [Option C]
       D) [Option D]
       ANSWER_LINE: [Option Letter] ||| [Detailed Explanation]

    3. NOTE ON MATH: You may use basic LaTeX (like $v_{{cm}}$ or $\\omega$), but keep it simple.
    """

    print(f"Drafting Paper with {model_name}...")
    response = llm.invoke(prompt)
    raw_content = response.content

    # --- Robust List-to-String Conversion ---
    raw_text = ""
    if isinstance(raw_content, str):
        raw_text = raw_content
    elif isinstance(raw_content, list):
        for item in raw_content:
            if isinstance(item, dict) and 'text' in item:
                raw_text += item['text']
            elif isinstance(item, str):
                raw_text += item
            else:
                raw_text += str(item)
    else:
        raw_text = str(raw_content)

    if raw_text.strip().startswith("{") and "'text':" in raw_text:
        try:
            parsed = ast.literal_eval(raw_text)
            if isinstance(parsed, dict) and 'text' in parsed:
                raw_text = parsed['text']
        except:
            pass

    # 4. Parse & Clean Content
    questions_data = []

    # Split by Q1., Q2., etc.
    raw_blocks = re.split(r'(Q\d+\.)', raw_text)

    for i in range(1, len(raw_blocks), 2):
        q_label = raw_blocks[i]
        q_body = raw_blocks[i+1]

        if "ANSWER_LINE:" in q_body:
            body_parts = q_body.split("ANSWER_LINE:")
            main_text = body_parts[0].strip()
            ans_part = body_parts[1].strip()

            if "|||" in ans_part:
                ans_letter, ans_expl = ans_part.split("|||", 1)
            else:
                ans_letter, ans_expl = ans_part, ""

            clean_main_text = clean_latex_to_reportlab(main_text)
            clean_expl = clean_latex_to_reportlab(ans_expl)

            questions_data.append({
                "label": q_label,
                "text": clean_main_text,
                "correct": ans_letter.strip(),
                "explanation": clean_expl.strip()
            })

    # 5. Build PDF with LaTeX Styling
    pdf_filename = f"Exam_Paper_{topic if topic else 'General'}.pdf"
    pdf_path = os.path.join(output_dir, pdf_filename)

    # We use A4 to match the latex template geometry
    doc = SimpleDocTemplate(
        pdf_path,
        pagesize=A4,
        rightMargin=1*inch, leftMargin=1*inch,
        topMargin=1.5*inch, bottomMargin=1*inch # Top margin larger to fit Header
    )

    # -- STYLES (Times-Roman to mimic LaTeX default) --
    styles = getSampleStyleSheet()

    # Title (Huge, Bold, Centered)
    title_style = ParagraphStyle(
        'LatexTitle',
        parent=styles['Heading1'],
        fontName='Times-Bold',
        fontSize=24,
        leading=28,
        alignment=TA_CENTER,
        spaceAfter=10
    )

    # Subtitle (Large, Centered)
    subtitle_style = ParagraphStyle(
        'LatexSubtitle',
        parent=styles['Normal'],
        fontName='Times-Roman',
        fontSize=16,
        leading=20,
        alignment=TA_CENTER,
        spaceAfter=5
    )

    # Date Style
    date_style = ParagraphStyle(
        'LatexDate',
        parent=styles['Normal'],
        fontName='Times-Roman',
        fontSize=12,
        alignment=TA_CENTER,
        spaceAfter=20
    )

    # Question Body
    q_style = ParagraphStyle(
        'QuestionStyle',
        parent=styles['Normal'],
        fontName='Times-Roman',
        fontSize=12,
        leading=15,
        spaceAfter=10
    )

    # Answer/Explanation Text
    ans_text_style = ParagraphStyle(
        'AnsText',
        parent=styles['Normal'],
        fontName='Times-Roman',
        fontSize=11,
        leading=13,
        spaceAfter=8
    )

    # -- Horizontal Line Helper --
    def draw_hrule():
        d = Drawing(450, 1)
        d.add(Line(0, 0, 450, 0))
        return d

    story = []

    # --- Title Block ---
    story.append(Paragraph("Examination Paper", title_style))
    story.append(Paragraph(f"Topic: {topic if topic else 'Full Syllabus'}", subtitle_style))
    current_date = datetime.now().strftime("%B %d, %Y")
    story.append(Paragraph(current_date, date_style))

    # Horizontal Rule
    story.append(draw_hrule())
    story.append(Spacer(1, 20))

    # --- Questions ---
    for q in questions_data:
        # Replace newlines with breaks for options
        formatted_body = q['text'].strip().replace("\n", "<br/>")

        # Combine Label and Text
        story.append(Paragraph(f"<b>{q['label']}</b> {formatted_body}", q_style))
        story.append(Spacer(1, 10))

    # --- Answer Key Page ---
    story.append(PageBreak())

    story.append(Paragraph("Answer Key", title_style))
    story.append(draw_hrule())
    story.append(Spacer(1, 15))

    for q in questions_data:
        q_num = q['label']
        ans = q['correct']
        expl = q['explanation']

        text = f"<b>{q_num}</b> Correct Option: <b>{ans}</b><br/><i>{expl}</i>"
        story.append(Paragraph(text, ans_text_style))
        story.append(Spacer(1, 6))

    # Build with Header/Footer Callback
    doc.build(story, onFirstPage=header_footer, onLaterPages=header_footer)

    print(f"✅ Exam Paper generated at: {pdf_path}")
    return pdf_path

# --- Execution ---
# Assumes 'output_dir' is defined in your environment (e.g., in Colab)
if 'output_dir' not in locals():
    output_dir = "generated_paper" # Default backup
    os.makedirs(output_dir, exist_ok=True)

topic = input("Enter your preferred topic (leave blank if want paper based on whole PDF):").strip()
if not topic:
  topic = None

num_questions = input("Enter your preferred number of questions (leave blank for default 5):").strip()
if not num_questions:
  num_questions = 5

pdf_path = generate_professional_mcq(output_dir, topic=topic, num_questions=num_questions)